# Robust SVM (WORK IN PROGRESS)

Support Vector Machines (SVMs) are a set of supervised learning methods primarily used for classification. A Robust SVM is a variation of the standard SVM designed to handle data that is noisy or contains outliers more effectively. While the standard SVM focuses on finding a hyperplane that best separates classes by maximizing the margin between them, a Robust SVM aims to improve the model’s performance and stability in the presence of noise and outliers.

In this notebook, we will consider example 4.5 introduced in Ali, Kolter, Diamond, and Boyd (2015) [1]. Based on the data given, we can reformulate the optimization problem as:

$$
\begin{aligned}
& \text{minimize} \quad \frac{1}{2} \| w \|^2 + C * \vec{1}^T \vec{\xi} \\
& \text{subject to:} \\
& w^T(x_i + u_x) + b \geq 1 - \xi _i\\
&  w^T (x_i + u_y) + b \geq 1 - \xi_i, \\
& \xi_i \geq 0

\end{aligned}
$$

In [1]:
import numpy as np
import cvxpy as cp
import lropt
from scipy.sparse import SparseEfficiencyWarning
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cvxpy')
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)

In [2]:
np.random.seed(1)
N=10
M = 10
MU = np.ones(N)
SIG = 0.1*np.eye(N)
DATA = np.random.multivariate_normal(MU, SIG, M)
y = np.random.choice([-1, 1], M)
C = 0.1

In [3]:
np.random.seed(1)
w = cp.Variable(N)
b = cp.Variable()
xi = cp.Variable(N)
u = lropt.UncertainParameter(M, uncertainty_set = lropt.Ellipsoidal(p=2,rho=0.5))


In [4]:
objective = cp.Minimize(cp.norm2(w) + C*(cp.sum(xi)))

In [5]:
constraints = []

for i in range(M):
    constraints.append(w@(DATA[i] + u) + b >= 1 - xi[i])
    constraints.append(-(w@(DATA[i] + u) + b) >= 1 - xi[i])

prob = lropt.RobustProblem(objective, constraints)
prob.solve()

In [6]:
b.value

array(-1.787127e-19)

## References

1. Ali, A., Kolter, J. Z., Diamond, S., & Boyd, S. (2015). Disciplined convex stochastic programming: A new framework for stochastic optimization. Proceedings of the 31st Conference on Uncertainty in Artificial Intelligence.
